<a href="https://colab.research.google.com/github/Husky-69/Testing/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pandas beautifulsoup4 lxml

In [13]:
# Known town-to-county mappings (public knowledge)
town_to_county = {
    # Aw - Tropical savanna
    'Isiolo': 'Isiolo',
    'Malindi': 'Kilifi',
    'Talek': 'Narok',
    'Amboseli': 'Kajiado',

    # BSh - Hot semi-arid
    'Archers Post': 'Samburu',
    'Mutomo': 'Kitui',
    'Voi': 'Taita Taveta',

    # Af - Tropical rainforest
    'Kisumu': 'Kisumu',
    'Kakamega': 'Kakamega',
    'Busia': 'Busia',
    'Kisii': 'Kisii',
    'Siaya': 'Siaya',

    # Am - Tropical monsoon
    'Bungoma': 'Bungoma',
    'Webuye': 'Bungoma',

    # Cfb - Oceanic (Highlands)
    'Eldoret': 'Uasin Gishu',
    'Iten': 'Elgeyo-Marakwet',
    'Nyeri': 'Nyeri',

    # Cwb - Subtropical highland
    'Nairobi': 'Nairobi',
    'Ngong': 'Kajiado',
    'Wundanyi': 'Taita Taveta',

    # BWh - Hot desert
    'Garissa': 'Garissa',
    'Lodwar': 'Turkana',

    # Csb - Warm-summer Med.
    'Nanyuki': 'Laikipia',
    'Meru': 'Meru',
    'Thika': 'Kiambu',

    # Cwa - Humid subtropical
    'Kimana': 'Kajiado',
}

In [14]:
# Initialize county climate map
county_climate = {}

# Known climate zones from Climate-Data.org
climate_zones = {
    'Aw': ['Isiolo', 'Malindi', 'Talek', 'Amboseli'],
    'BSh': ['Archers Post', 'Mutomo', 'Voi'],
    'Af': ['Kisumu', 'Kakamega', 'Busia', 'Kisii', 'Siaya'],
    'Am': ['Bungoma', 'Webuye'],
    'Cfb': ['Eldoret', 'Iten', 'Nyeri'],
    'Cwb': ['Nairobi', 'Ngong', 'Wundanyi'],
    'BWh': ['Garissa', 'Lodwar'],
    'Csb': ['Nanyuki', 'Meru', 'Thika'],
    'Cwa': ['Kimana']
}

# Map towns to counties and assign climate
for climate_class, towns in climate_zones.items():
    for town in towns:
        if town in town_to_county:
            county = town_to_county[town]
            county_climate[county] = climate_class

# Fill missing counties with dominant class: Aw (Tropical Savanna)
all_counties = [
    "Baringo", "Bomet", "Bungoma", "Busia", "Elegeyo-Marakwet", "Embu", "Garissa",
    "Homa Bay", "Isiolo", "Kajiado", "Kakamega", "Kericho", "Kiambu", "Kilifi",
    "Kirinyaga", "Kisii", "Kisumu", "Kitui", "Kwale", "Laikipia", "Lamu", "Machakos",
    "Makueni", "Mandera", "Marsabit", "Meru", "Migori", "Mombasa", "Murang'a",
    "Nairobi", "Nakuru", "Nandi", "Narok", "Nyamira", "Nyandarua", "Nyeri",
    "Samburu", "Siaya", "Taita Taveta", "Tana River", "Tharaka-Nithi", "Trans Nzoia",
    "Turkana", "Uasin Gishu", "Vihiga", "Wajir", "West Pokot"
]

# Assign 'Aw' as default (per Climate-Data.org: "dominated by Aw")
final_climate_map = {}
for county in all_counties:
    final_climate_map[county] = county_climate.get(county, 'Aw')  # default = Aw

# Convert to DataFrame
import pandas as pd
climate_df = pd.DataFrame(list(final_climate_map.items()), columns=['county', 'climate_class'])

In [15]:
def assign_risk(climate_class):
    if climate_class in ['BWh', 'BSh']:
        return "Drought"
    elif climate_class in ['Af', 'Am']:
        return "Flooding & Pests"
    elif climate_class == 'Aw':
        return "Erratic Rains & Drought"
    elif climate_class in ['Cfb', 'Cwb', 'Csb']:
        return "Frost & Erratic Rains"
    elif climate_class == 'Cwa':
        return "Mixed: Frost & Rain Variability"
    else:
        return "Erratic Rains & Drought"  # default

climate_df['primary_risk'] = climate_df['climate_class'].apply(assign_risk)
climate_df.head(10)

,county,climate_class,primary_risk
0,Baringo,Aw,Erratic Rains & Drought
1,Bomet,Aw,Erratic Rains & Drought
2,Bungoma,Am,Flooding & Pests
3,Busia,Af,Flooding & Pests
4,Elegeyo-Marakwet,Aw,Erratic Rains & Drought
5,Embu,Aw,Erratic Rains & Drought
6,Garissa,BWh,Drought
7,Homa Bay,Aw,Erratic Rains & Drought
8,Isiolo,Aw,Erratic Rains & Drought
9,Kajiado,Cwa,Mixed: Frost & Rain Variability


In [17]:
# In your code, you can now create a "Lead Farmer" registry
import numpy as np

lead_farmers = []
for county in climate_df['county']:
    # Simulate assigning 1 lead farmer per sub-county
    for i in range(1):  # Simplified: 1 per county for now
        lead_farmers.append({
            'county': county,
            'climate_class': climate_df[climate_df['county'] == county]['climate_class'].iloc[0],
            'primary_risk': climate_df[climate_df['county'] == county]['primary_risk'].iloc[0],
            'hub_id': f"{county}_LF_{i+1}",
            'phone_model': "Samsung Galaxy A12",  # Assumed "good phone"
            'agent_age_group': "Youth",  # Target 15–30 yr olds
            'agent_gender': "Male" if np.random.rand() > 0.3 else "Female"  # Reflects HABITABLE gender findings
        })

lead_farmer_df = pd.DataFrame(lead_farmers)
print("Lead Farmer Network:", len(lead_farmer_df), "units")
lead_farmer_df.head()

Lead Farmer Network: 47 units


,county,climate_class,primary_risk,hub_id,phone_model,agent_age_group,agent_gender
0,Baringo,Aw,Erratic Rains & Drought,Baringo_LF_1,Samsung Galaxy A12,Youth,Female
1,Bomet,Aw,Erratic Rains & Drought,Bomet_LF_1,Samsung Galaxy A12,Youth,Male
2,Bungoma,Am,Flooding & Pests,Bungoma_LF_1,Samsung Galaxy A12,Youth,Female
3,Busia,Af,Flooding & Pests,Busia_LF_1,Samsung Galaxy A12,Youth,Male
4,Elegeyo-Marakwet,Aw,Erratic Rains & Drought,Elegeyo-Marakwet_LF_1,Samsung Galaxy A12,Youth,Female


In [18]:
# Define alert templates by risk
alert_templates = {
    "Drought": "🚨 DROUGHT ALERT: Soil dry ({soil_moisture}% moisture). Consider moving livestock or buying feed. -M-Kulima IoT",
    "Flooding & Pests": "🌧️ FLOOD RISK: Drain fields. Monitor for pests like Fall Armyworm. -M-Kulima",
    "Erratic Rains & Drought": "⚠️ Weather Warning: Rainfall unpredictable. Plant drought-tolerant maize. -M-Kulima",
    "Frost & Erratic Rains": "❄️ FROST WARNING: Protect young crops. Delay planting. -M-Kulima",
    "Mixed: Frost & Rain Variability": "⚠️ Mixed Risk: Watch for frost at night, rain during day. -M-Kulima"
}

# Generate sample alerts for Lead Farmers
def generate_alert(row):
    template = alert_templates.get(row['primary_risk'], "✅ Normal conditions. Proceed with planting schedule.")
    return template.format(soil_moisture=25)  # Simulate real sensor reading

lead_farmer_df['default_alert'] = lead_farmer_df.apply(generate_alert, axis=1)
lead_farmer_df[['county', 'primary_risk', 'default_alert']].head(5)

,county,primary_risk,default_alert
0,Baringo,Erratic Rains & Drought,⚠️ Weather Warning: Rainfall unpredictable. Pl...
1,Bomet,Erratic Rains & Drought,⚠️ Weather Warning: Rainfall unpredictable. Pl...
2,Bungoma,Flooding & Pests,🌧️ FLOOD RISK: Drain fields. Monitor for pests...
3,Busia,Flooding & Pests,🌧️ FLOOD RISK: Drain fields. Monitor for pests...
4,Elegeyo-Marakwet,Erratic Rains & Drought,⚠️ Weather Warning: Rainfall unpredictable. Pl...


In [20]:
# Simulate sending an SMS via Africa's Talking (pseudo-code)
def send_sms(phone_number, message):
    # In production: use Africa's Talking SDK
    print(f"📤 SMS sent to {phone_number}: {message}")

# Example: Send alert to 5 farmers in Baringo
baringo_farmers = ["+254700123456", "+254700123457", "+254700123458"]
alert = lead_farmer_df[lead_farmer_df['county'] == 'Baringo']['default_alert'].iloc[0]

for phone in baringo_farmers:
    send_sms(phone, alert)

📤 SMS sent to +254700123456: ⚠️ Weather Warning: Rainfall unpredictable. Plant drought-tolerant maize. -M-Kulima
📤 SMS sent to +254700123457: ⚠️ Weather Warning: Rainfall unpredictable. Plant drought-tolerant maize. -M-Kulima
📤 SMS sent to +254700123458: ⚠️ Weather Warning: Rainfall unpredictable. Plant drought-tolerant maize. -M-Kulima


In [21]:
# Simulate a "Credit Score" based on activity
def calculate_credit_score(activity_log):
    # Simple model: 1 point per completed task, 5 points for using IoT sensor
    score = sum([1 for act in activity_log if act == 'task_completed']) + \
            (5 if 'sensor_used' in activity_log else 0)
    return min(score, 100)  # Max score 100

# Example: Lead Farmer in Garissa (Drought zone) has high activity
activity_log = ['task_completed', 'sensor_used', 'task_completed']
credit_score = calculate_credit_score(activity_log)
print(f"📊 Credit Score: {credit_score}/100 → Eligible for KES 5,000 loan")

📊 Credit Score: 7/100 → Eligible for KES 5,000 loan


In [22]:
# Placeholder for image collection plan
image_collection_plan = {
    "county": "Kakamega",
    "pest": "Fall Armyworm",
    "target_images": 100,
    "collection_method": "Lead Farmer takes photos with phone camera",
    "labeling": "Partner with local agronomist for ground truth",
    "model": "MobileNetV2 (TF Lite) fine-tuned on local data"
}

print("📸 Image Collection Plan:")
for key, value in image_collection_plan.items():
    print(f"  • {key}: {value}")

📸 Image Collection Plan:
  • county: Kakamega
  • pest: Fall Armyworm
  • target_images: 100
  • collection_method: Lead Farmer takes photos with phone camera
  • labeling: Partner with local agronomist for ground truth
  • model: MobileNetV2 (TF Lite) fine-tuned on local data


In [23]:
# Prioritize counties for Phase 1 rollout
climate_df['priority'] = np.where(
    climate_df['primary_risk'].isin(['Drought', 'Flooding & Pests']),
    'High',
    'Medium'
)

phase_1_counties = climate_df[climate_df['priority'] == 'High']['county'].tolist()
print("🚀 Phase 1 Rollout (High Priority):", len(phase_1_counties), "counties")
print(phase_1_counties[:10])  # Show first 10

🚀 Phase 1 Rollout (High Priority): 10 counties
['Bungoma', 'Busia', 'Garissa', 'Kakamega', 'Kisii', 'Kisumu', 'Kitui', 'Samburu', 'Siaya', 'Turkana']
